<a href="https://colab.research.google.com/github/lingduoduo/NLP/blob/master/02_agents_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAIAgent

`OpenAIAgent` is an OpenAI (function calling) Agent. It uses the OpenAI function API to reason about whether to use a tool, and returning the response to the user. It supports both a flat list of tools as well as retrieval over the tools.

LlamaIndex notebook: https://docs.llamaindex.ai/en/stable/examples/agent/openai_agent_with_query_engine.html.

## Step 1: Install and Setup

In [1]:
!pip install -q llama_index pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.6 MB/s eta 0:00:00


In [2]:
import logging, sys, os
import nest_asyncio
from google.colab import userdata

# set OpenAI API key in environment variable
os.environ["OPENAI_API_KEY"] = "sk-"

# serves to enable nested asynchronous event loops, recommended for colab notebook
nest_asyncio.apply()

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
!mkdir sample_data/reports
!wget https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2020/executive-summary-2020.pdf -O sample_data/reports/2020-executive-summary.pdf
!wget https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2021/executive-summary-2021.pdf -O sample_data/reports/2021-executive-summary.pdf
!wget https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2022/executive-summary-2022.pdf -O sample_data/reports/2022-executive-summary.pdf

--2024-07-14 23:44:14--  https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2020/executive-summary-2020.pdf
Resolving www.fiscal.treasury.gov (www.fiscal.treasury.gov)... 164.95.95.168, 2610:108:3100:100c::8:118
Connecting to www.fiscal.treasury.gov (www.fiscal.treasury.gov)|164.95.95.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2323072 (2.2M) [application/pdf]
Saving to: ‘sample_data/reports/2020-executive-summary.pdf’

sample_data/reports 100%[===================>]   2.21M  1.87MB/s    in 1.2s    

2024-07-14 23:44:16 (1.87 MB/s) - ‘sample_data/reports/2020-executive-summary.pdf’ saved [2323072/2323072]

--2024-07-14 23:44:16--  https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2021/executive-summary-2021.pdf
Resolving www.fiscal.treasury.gov (www.fiscal.treasury.gov)... 164.95.95.168, 2610:108:3100:100c::8:118
Connecting to www.fiscal.treasury.gov (www.fiscal.treasury.gov)|164.95.95.168|:443... connected

## Step 2: Load data, build indices, define OpenAIAgent

In [5]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.agent.openai import OpenAIAgent
import os

query_engine_tools = []

for filename in os.listdir("sample_data/"):
    if filename.endswith(".pdf"):
        file_path = os.path.join("reports", filename)

        with open(file_path, "r") as file:
            documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
            print(f"Loaded {len(documents)} documents from {filename}")
            print(filename[:-4])

            index = VectorStoreIndex.from_documents(documents)
            query_engine = index.as_query_engine(similarity_top_k=5)
            query_engine_tool = QueryEngineTool.from_defaults(
                query_engine=query_engine,
                name=f"{filename[:-4]}",  # Construct name without extension
                description=f"Provides information about the U.S. government financial report {filename[:-4]}",
            )
            query_engine_tools.append(query_engine_tool)

agent = OpenAIAgent.from_tools(query_engine_tools, verbose=True)

## Step 3: Execute Queries

In [6]:
from IPython.display import Markdown

response = agent.chat("Can you compare and contrast the government's bottom line net operating cost amount for all three years and tell me which year has the highest cost?")
display(Markdown(f"<b>{response}</b>"))

Added user message to memory: Can you compare and contrast the government's bottom line net operating cost amount for all three years and tell me which year has the highest cost?


<b>To compare and contrast the government's bottom line net operating cost amount for all three years, we would need the specific figures for each year. Without that information, we cannot determine which year has the highest cost.</b>